In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogatePerformanceEstimator import SurrogatePerformanceEstimator

In [2]:
train_instances, test_instances = TSP_train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=3,
)
train_instances

InstanceList(size=3)[TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp)]

In [3]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=1)
portfolio

Portfolio(size=1)[Solver(id=52980316141715150)]

In [4]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    t_c=600,
    calculate_features=True,
    estimator=None,
    estimator_pct=0.5,
)

In [5]:
first_model_iter = 50
last_model_iter = 0

estimator = None
db = DB()

for _ in aac.configure_iter():
    if aac.iter >= first_model_iter and aac.iter - last_model_iter >= 5:
        X, y = get_model_training_data(db)
        estimator = SurrogatePerformanceEstimator(max_cost=TSP_LKH_Solver.MAX_COST)
        estimator.fit(X, y)
        estimator.log()
        last_model_iter = aac.iter
        aac.update(estimator=estimator)

[2024-12-28 00:17:21] DEBUG     AAC(prefix=config, iter=1, configuration_time=[600.00])
[2024-12-28 00:17:21] DEBUG     Portfolio.evaluate(config;aac_iter=1)
[2024-12-28 00:17:21] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2024-12-28 00:17:21] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-28 00:17:21] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-28 00:17:27] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp))
[2024-12-28 00:17:27] DEBUG     calculate_features(instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp))
[2024-12-28 00:17:30] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp))
[2024-12-28 

In [6]:
db = DB()
# db = DB("database/run-test_923830.db")

In [14]:
X, y = get_model_training_data(db)
model = SurrogatePerformanceEstimator(max_cost=TSP_LKH_Solver.MAX_COST)
# model.fit(X, y)

In [16]:
from sklearn.utils.validation import check_is_fitted
check_is_fitted(model._DEFAULT_CLASSIFIER)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [12]:
model.predict(X)

array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100.])

In [5]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,...,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,855157651668393360,TSP/CEPS_benchmark/explosion/21.tsp,14928292.0,0.006290,1.484946,1.522267,3.139028,0.916031,3.132738,0.601754,...,0.125000,0.500000,0.5,0.500000,0.875000,0.90125,0.20000,0.21,450.566621,0.68
1,1353678304570487518,TSP/CEPS_benchmark/rotation/34.tsp,15486460.0,0.000684,1.495912,1.523094,3.138384,0.912371,3.137700,0.599025,...,0.111111,0.500000,0.5,0.500000,0.888889,0.87750,0.24750,0.08,394.965129,0.42
2,810870798045289410,TSP/CEPS_benchmark/compression/09.tsp,16003060.0,0.000000,1.417332,1.483901,3.141593,1.026220,3.141593,0.691569,...,0.166667,0.500000,0.5,0.500000,0.833333,0.87125,0.26500,0.15,431.220045,0.72
3,1306299629087479721,TSP/CEPS_benchmark/implosion/34.tsp,19968838.0,0.000395,1.483846,1.518058,3.138849,0.916994,3.138454,0.604057,...,0.125000,0.333333,0.5,0.666667,0.875000,0.86500,0.27375,0.10,400.640868,0.42
4,410041616729589825,TSP/CEPS_benchmark/linearprojection/25.tsp,13578491.0,0.009872,1.625140,1.570276,3.131472,0.931936,3.121600,0.593485,...,0.250000,0.500000,0.5,0.500000,0.750000,0.87500,0.25500,0.16,382.995621,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,707351994282137341,TSP/CEPS_benchmark/cluster/19.tsp,19287084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,540324668286432217,TSP/CEPS_benchmark/uniform_portgen/27.tsp,20825913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1765803814609198146,TSP/CEPS_benchmark/grid/00.tsp,21258773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1378924496111515287,TSP/CEPS_benchmark/explosion/32.tsp,12927140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.00,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,287011037061967493,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,0.00,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
2,176228367859327934,0.85,1.0,0.0,0.0,0.4,0.0,0.0,1.0,0.50,...,0.354664,0.333333,0.25,0.8,0.4,0.408163,1.0,1.0,2.0,0.0
3,2041120758543581287,0.80,1.0,0.0,2.0,0.7,0.0,1.0,0.0,0.25,...,0.365942,1.000000,0.25,0.6,0.8,0.051020,0.0,1.0,5.0,0.0
4,1056464823932360013,0.65,1.0,1.0,2.0,0.0,0.0,0.0,1.0,1.00,...,0.322513,0.666667,1.00,0.8,0.8,0.112245,0.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,1543248257296619305,0.00,0.0,0.0,2.0,0.6,0.0,1.0,1.0,0.50,...,0.713917,1.000000,0.25,0.6,0.8,0.693878,1.0,1.0,1.0,1.0
77,1997176499706663462,0.45,0.0,1.0,3.0,0.3,0.0,0.0,1.0,1.00,...,0.497030,0.555556,1.00,1.0,0.8,0.408163,1.0,1.0,3.0,1.0
78,581181491700982781,0.05,1.0,1.0,3.0,0.6,0.0,0.0,0.0,0.75,...,0.515792,0.222222,0.75,0.0,0.6,0.102041,1.0,0.0,3.0,1.0
79,16557392879338281,0.55,0.0,1.0,1.0,0.2,0.0,0.0,1.0,0.50,...,0.686544,0.777778,0.00,0.0,0.8,0.673469,0.0,1.0,1.0,0.0


In [10]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,config;aac_iter=1_52980316141715150_8551576516...,config;aac_iter=1,52980316141715150,855157651668393360,100.00,23.448853,0,0,0
1,config;aac_iter=1_52980316141715150_1353678304...,config;aac_iter=1,52980316141715150,1353678304570487518,100.00,15.938080,0,0,0
2,config;aac_iter=1_52980316141715150_1306299629...,config;aac_iter=1,52980316141715150,1306299629087479721,3.04,8.142928,0,0,0
3,config;aac_iter=1_52980316141715150_8108707980...,config;aac_iter=1,52980316141715150,810870798045289410,100.00,16.167195,0,0,0
4,config;aac_iter=1_52980316141715150_4100416167...,config;aac_iter=1,52980316141715150,410041616729589825,100.00,17.013006,0,0,0
...,...,...,...,...,...,...,...,...,...
495,test_2015280438004251377_1667136710979054795,test,2015280438004251377,1667136710979054795,100.00,10.000000,0,0,0
496,test_2015280438004251377_707351994282137341,test,2015280438004251377,707351994282137341,100.00,10.000000,0,0,0
497,test_2015280438004251377_1765803814609198146,test,2015280438004251377,1765803814609198146,2.71,2.710000,0,0,0
498,test_2015280438004251377_366756074013316631,test,2015280438004251377,366756074013316631,5.29,5.290000,0,0,0


In [11]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)

,id,solver_id,instance_id,cost
0,52980316141715150_855157651668393360,52980316141715150,855157651668393360,100.00
1,52980316141715150_1353678304570487518,52980316141715150,1353678304570487518,100.00
2,52980316141715150_1306299629087479721,52980316141715150,1306299629087479721,3.04
3,52980316141715150_810870798045289410,52980316141715150,810870798045289410,100.00
4,52980316141715150_410041616729589825,52980316141715150,410041616729589825,100.00
...,...,...,...,...
495,2015280438004251377_1667136710979054795,2015280438004251377,1667136710979054795,100.00
496,2015280438004251377_707351994282137341,2015280438004251377,707351994282137341,100.00
497,2015280438004251377_1765803814609198146,2015280438004251377,1765803814609198146,2.71
498,2015280438004251377_366756074013316631,2015280438004251377,366756074013316631,5.29
